Rewirte the p function so that it is now polymorphic with respect to the number of arguments. The last argument is always either a set or a probability distribution dictionary. The other arguments are eithers sets or predicate functions.

The p function be able to run like that: 

p(a_1, a_2, a_3, ..., MM)

where the probability is p (a_1| a_2, a_3, ...) operating on either the set or the dictionary MM.

This is a group assignment and each group submit one jupyter notebook file. Name your file as Assignment 7_GroupNum.

****Assignment requirement:****   
Below is the original p function.
```python
def p(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs."""
    # branch on the type of the first argument
    if is_predicate(event):
        # transform the mapping (untangible) 'event' into the collection (tangible) 'event'
        event = such_that(event, space)
        
    if isinstance(space, ProbDist):
        # if space is a dictionary of distinct probabilities, where each item does not count as the same amount
        return sum(space[o] for o in space if o in event)
    else:
        # space is not a dictionary but a collection, let's fall back to our original division
        return Fraction(len(event & space), len(space))

is_predicate = callable

def such_that(predicate, space): 
    """The outcomes in the sample space for which the predicate is true.
    If space is a set, return a subset {outcome,...} with outcomes where predicate(element) is true;
    if space is a ProbDist, return a ProbDist {outcome: frequency,...} with outcomes where predicate(element) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}
```

And now p has to be upgraded to p(a_1, a_2, a_3, ..., MM), where a_1,_a_2,a_3 are either sets or predicate functions.

In [33]:
class ProbDist(dict):
    """A Probability Distribution; an {outcome: probability} mapping."""
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        # Make probabilities sum to 1.0; assert no negative probabilities
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0
            

In [42]:

from fractions import Fraction
def such_that(predicate, space): 
    """The outcomes in the sample pace for which the predicate is true.
    If space is a set, return a subset {outcome,...} with outcomes where predicate(element) is true;
    if space is a ProbDist, return a ProbDist {outcome: frequency,...} with outcomes where predicate(element) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}

In [51]:
is_predicate = callable
def p(event, *args): 
    """The probability of an event, given a sample space "space" of exclusive outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs."""
    ##########   Step 1: Get the space and the prior_events  #################
    # the last entry of args is the sample space
    space = args[-1]
    # the rest of the entry in the args are kept and assigned to a renewed tuple to form the prior_events
    args = tuple(args[i] for i in range(len(args)-1))
    ##########   Step 2: Initialize prior_events as the entire sampling space   ############ 
    # For simplification, prior_events will only keep the favorable event names, not the probabilities
    if isinstance(space, ProbDist):
        prior_events=tuple(space.keys())
    else:
        prior_events=space
    ##########   Step 3: build an event collection for the prior events   ############ 
    # prior_events will take the intersection of all the filters in args
    # loop through args to filter the original space, in order to build the prior events space
    for argument in args: 
        # transform any mapping (untangible) 'event' in the unnamed predicates into the collection (tangible) 'event'
        if is_predicate(argument): 
            prior_events=[i for i in prior_events if argument(i)]
        else:
            prior_events=[i for i in prior_events if i in argument]
    ##########   Step 4: if needed, make prior_events the same type of ProbDist, each outcome will retrieve its probability ##########
    if isinstance(space, ProbDist):prior_events=such_that(lambda x:x in prior_events,space)
    ##########   Step 5: make event the same type of prior_events, each outcome will retrieve its probability ##########
    # branch on the type of the first argument
    if is_predicate(event):
        # transform the mapping (untangible) 'event' into the collection (tangible) 'event'
        event = such_that(event, prior_events)
    ##########   Step 6: Calculate the conditional probability ##########
    if isinstance(prior_events, ProbDist):
        # if space is a dictionary of distinct probabilities, where each item does not count as the same amount
        return sum(prior_events[o] for o in prior_events if o in event)
    else:
        # space is not a dictionary but a collection, let's fall back to our original division
        return Fraction(len(event & prior_events), len(prior_events))

Will this new p(a_1|a_2,a_3...) work? let's test!

In [52]:
# Representing the probability distribution of bag94 and bag96 by passing in respective distributions in the form of dictionaries of each year
bag94 = ProbDist(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
bag96 = ProbDist(blue = 24, green = 20, orange = 16, yellow = 14, red = 13, brown = 13)  
bag94

{'brown': 0.3,
 'yellow': 0.2,
 'red': 0.2,
 'green': 0.1,
 'orange': 0.1,
 'tan': 0.1}

In [45]:
# We now define the universal sample space MM as a joint distribution of 94-96. In other words, one M&M would be picked from bag94 and one from bag96.
# Outcome 'yellow green' means that yellow is picked from bag94 and green is picked from bag96
# Here, we will use set comprehension 
# Here, we will get to know the universal sample space MM

def joint(A, B, sep=''):
    """The joint distribution of two independent probability distributions. 
    Result is all entries of the form {a+sep+b: P(a)*P(b)}"""
    return ProbDist({a + sep + b: A[a] * B[b] 
                        for a in A 
                        for b in B})

MM = joint(bag94, bag96, ' ')
MM                  #universal sample space

{'brown blue': 0.07199999999999997,
 'brown green': 0.05999999999999997,
 'brown orange': 0.04799999999999998,
 'brown yellow': 0.04199999999999998,
 'brown red': 0.038999999999999986,
 'brown brown': 0.038999999999999986,
 'yellow blue': 0.04799999999999998,
 'yellow green': 0.03999999999999999,
 'yellow orange': 0.03199999999999999,
 'yellow yellow': 0.02799999999999999,
 'yellow red': 0.025999999999999992,
 'yellow brown': 0.025999999999999992,
 'red blue': 0.04799999999999998,
 'red green': 0.03999999999999999,
 'red orange': 0.03199999999999999,
 'red yellow': 0.02799999999999999,
 'red red': 0.025999999999999992,
 'red brown': 0.025999999999999992,
 'green blue': 0.02399999999999999,
 'green green': 0.019999999999999993,
 'green orange': 0.015999999999999993,
 'green yellow': 0.013999999999999995,
 'green red': 0.012999999999999996,
 'green brown': 0.012999999999999996,
 'orange blue': 0.02399999999999999,
 'orange green': 0.019999999999999993,
 'orange orange': 0.015999999999999

In [53]:
# Lets take a look at one is yellow and one is green part.
# We now define a predicate function(a function that returns true for an outcome that is in the event) named yellow_and_green which would 
# return 'yellow' in outcome and 'green' in the outcome such that(the subset of universal sample space for which the predicate(yellow_and_green) = true) 
# Here, the sample space is a Probability Distribution. So it would return a ProbDist {outcome: frequency,...} with outcomes where predicate = true.
def yellow_and_green(outcome): return 'yellow' in outcome and 'green' in outcome
such_that(yellow_and_green,MM) 

{'yellow green': 0.7407407407407408, 'green yellow': 0.25925925925925924}

In [54]:
# Now to answer the question, what is the probability that the yellow M&M came from bag94? (since we don't know which came from which bag)
# we define a predicate function yellow94 which would return the outcome that starts with 'yellow'

def yellow94(outcome): return outcome.startswith('yellow')

In [55]:
# Now we implement conditional probability using the functions defined above.
# Probability of an event (picking of a yellow M&M from bag94) based on existing knowledge of predicate(yellow94) being true in the universal sample space(MM)

p(yellow94, such_that(yellow94, MM))

1.0

In [56]:
#  Probability of an event yellow94 (picking of a yellow M&M from bag94) based on existing knowledge of picking two M&Ms yellow and green 
#  in the begining being true in the universal sample space(MM)

p(yellow94,yellow_and_green, MM)

0.7407407407407408

##### This confirms there is a 74.07% chance of the picked M&M coming from bag94. This proves that our redefined polymorphic p function is working perfect!

Explanation of the p-function:

The original p (event, space) function takes 2 forms of "event": either a predicate (a callable function that examines the outcome name and returns True or False) or a collection (a set or dictionary of favorable outcomes). It also takes 2 forms of "space": a collection of universal sample space containing all the exclusive outcomes, either a set (when the experiment is equiprobable) or a dictionary (when the experiment is non-equiprobable, and the outcome has uneven probability distribution). 


Explanation of such_that function:

The role of "such_that" function has 2 folds of meaning: the 1st is to turn any predicate type of "event" into a collection; the 2nd is to normalize the probability distribution of the outcomes in event if the samples are dictionary type.  Since such_that function normalize the probability distribution, it is also good for returning an updated prior event. The asterisked argument of a function *args (also known as positional arguments) will receive arbitrary number of unnamed arguments, which allows a flexibility of input to the function. Likewise, **kwargs (also known as keyword arguments) receives as many named arguments as possible. 

Consider a simple example of positional arguments or *args:

def p (a, *b): 

    return b

p (1, 2, 3, 4, 5)		                    # here we pass a few integers

(2,3,4,5)		                            # Output returns a tuple of positional arguments with respect to b

Here 1 is passed to a, and the rest of the unnamed input arguments 2,3,4,5 are received by *b and become a tuple b=(2,3,4,5)


Now, consider a simple example of keyword arguments or *kwargs:

def p (a, **b): 

    return b

p (1, x=2, y=3, z=4, w=5)	                # here we pass a few integers to some variables 

{x:2, y:3, z:4, w:5}		                # Output returns a dictionary of key value pairs

Here 1 is passed to a, and the rest of the named input arguments x=2, y=3, z=4, w=5 is received by **b and became a dictionary {x:2, y:3, z:4, i:5} 

Explanation on conditional probability:

Now in this assignment, we are required p (a_1| a_2, a_3...) which means the conditional probability of a_1 given that events a_2, a_3 ... already happened. We need to take arbitrary predicates a_2, a_3 and forms a prior event space of a_1. So, we introduced a new variable named prior_events. It's an intermediate event space between universal sample space and the resultant event space. Hence, in this case we can make use of such_that function which would check if the predicate were true in the given universal sample space.


The requirement is that the last argument in p should be the universal sampling space (DK in Danish kid example or MM in M&M example). Therefore, we shall separate the args first. "args [-1]" is the last entry, recognized as space. To isolate the prior events, we use 
"args = tuple(args[i] for i in range(len(args)-1))" is the remaining entries, should be recognized as all the prior_events. It could be either predicates (functions) or collections. We prefer turning all of them into collections and find the intersection of all of them. We then transform any mapping (untangible) 'event' in the unnamed predicates into the collection (tangible) 'event' to calculate the conditional probability. The resultant space is a dictionary of distinct probabilities, where each item does not count as the same amount, we use           sum(prior_events[o] for o in prior_events if o in event) to get the resultant space.
